# Merging SSURGO and NHD

In [1]:
import pickle
import pandas as pd

with open("../2. Extract NHD parameters from NHD pickle files/2021.03.09_nhd_variables_extracted_from_2.5kmX2.5km_pkl3.pkl", "rb") as f:
    df_n_km = pickle.load(f)

with open("../2. Extract NHD parameters from NHD pickle files/2021.03.09_nhd_variables_extracted_from_200mX200m_pkl3.pkl", "rb") as f:
    df_n_m = pickle.load(f)
    
    
df_n_km.drop(["hydclprs", "aws025wta", "drclassdcd"], axis=1, inplace=True)
df_n_m.drop(["hydclprs", "aws025wta", "drclassdcd"], axis=1, inplace=True)
df_s = pd.read_csv("../../SSURGO/SSURGO_combined_regular_clean_with_ssurgo_variables_updated.csv")

In [2]:
ssurgo_vars = ['Index', 'hydclprs', 'aws025wta', 'drclassdcd', 'slopegradd', 'wtdepannmi',
       'flodfreqdc', 'pondfreqpr', 'wtdepaprju']
df_s_filtered = df_s[ssurgo_vars]
# df_s_filtered

In [3]:
df_n_km["Index_"] = df_n_km.Index
df_n_m["Index_"] = df_n_m.Index
# df_n_km.columns

In [4]:
untouchable_cols = ['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'mukey', 'Index_']
touchable_cols = list(set(df_n_km.columns) - set(untouchable_cols))
# touchable_cols

In [5]:
touchable_cols = [col for col in touchable_cols]

In [6]:
# append this part at the very end
df_constant = df_n_km[untouchable_cols].copy()
df_constant = df_constant.rename(columns={"Index_" : "Index"})
df_constant.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination', 'cwa_determination',
       'rha1', 'rha2', 'cwa1', 'cwa2', 'cwa3', 'cwa4', 'cwa5', 'cwa6', 'cwa7',
       'cwa8', 'cwa9', 'potential_wetland', 'index', 'mukey', 'Index'],
      dtype='object')

In [7]:
for col in untouchable_cols:
    df_n_km.drop([str(col)], axis=1, inplace=True)
    df_n_m.drop([str(col)], axis=1, inplace=True)

In [8]:
n_km_cols = []
n_m_cols = []
for col in df_n_km.columns:
    if col != "Index":
        n_km_cols.append((str(col), str(col) + "_2500m"))
    else:
        n_km_cols.append((str(col), str(col)))
for col in df_n_m.columns:
    if col != "Index":
        n_m_cols.append((str(col), str(col) + "_200m"))
    else:
        n_m_cols.append((str(col), str(col)))
    
new_km_col_dict = dict(n_km_cols)    
new_m_col_dict = dict(n_m_cols)    
new_km_col_dict
df_n_km.rename(columns = new_km_col_dict, inplace=True)
df_n_m.rename(columns = new_m_col_dict, inplace=True)

In [9]:
df_n_km.columns

Index(['Index', 'nhd_vars_wb_2500m', 'nhd_vars_fl_2500m',
       'wb_comid_list_2500m', 'wb_ftype_str_list_2500m',
       'wb_gnis_id_list_2500m', 'wb_area_list_2500m', 'fl_comid_list_2500m',
       'fl_ftype_str_list_2500m', 'fl_gnis_id_list_2500m',
       'fl_length_list_2500m', 'fl_areasqkm_sum_2500m',
       'fl_areasqkm_count_2500m', 'fl_areasqkm_mean_2500m',
       'fl_gnis_name_ind_sum_2500m', 'fl_gnis_name_ind_count_2500m',
       'fl_gnis_name_ind_mean_2500m', 'fl_totdasqkm_sum_2500m',
       'fl_totdasqkm_count_2500m', 'fl_totdasqkm_mean_2500m',
       'fl_flow_type_sum_2500m', 'fl_flow_type_count_2500m',
       'fl_flow_type_mean_2500m', 'fl_streamorde_sum_2500m',
       'fl_streamorde_count_2500m', 'fl_streamorde_mean_2500m',
       'fl_intephem_sum_2500m', 'fl_intephem_count_2500m',
       'fl_intephem_mean_2500m', 'fl_startflag_sum_2500m',
       'fl_startflag_count_2500m', 'fl_startflag_mean_2500m',
       'fl_divergence_sum_2500m', 'fl_divergence_count_2500m',
       'f

In [10]:
df1 = df_n_m
df2 = df_n_km
df3 = df_s_filtered

df_merged = pd.merge(df1, df2, on="Index")
df_merged = pd.merge(df_merged, df3, on="Index")
df_merged = pd.merge(df_constant, df_merged, on="Index")
df_merged.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination',
       ...
       'fl_length_count_2500m', 'fl_length_mean_2500m', 'hydclprs',
       'aws025wta', 'drclassdcd', 'slopegradd', 'wtdepannmi', 'flodfreqdc',
       'pondfreqpr', 'wtdepaprju'],
      dtype='object', length=132)

In [11]:
for col in df_merged.columns:
  print(col)

Unnamed: 0
Unnamed: 0.1
jurisdiction_type
da_number
district
project_name
longitude
latitude
date_issued_or_denied
rha_determination
cwa_determination
rha1
rha2
cwa1
cwa2
cwa3
cwa4
cwa5
cwa6
cwa7
cwa8
cwa9
potential_wetland
index
mukey
Index
nhd_vars_wb_200m
nhd_vars_fl_200m
wb_comid_list_200m
wb_ftype_str_list_200m
wb_gnis_id_list_200m
wb_area_list_200m
fl_comid_list_200m
fl_ftype_str_list_200m
fl_gnis_id_list_200m
fl_length_list_200m
fl_areasqkm_sum_200m
fl_areasqkm_count_200m
fl_areasqkm_mean_200m
fl_gnis_name_ind_sum_200m
fl_gnis_name_ind_count_200m
fl_gnis_name_ind_mean_200m
fl_totdasqkm_sum_200m
fl_totdasqkm_count_200m
fl_totdasqkm_mean_200m
fl_flow_type_sum_200m
fl_flow_type_count_200m
fl_flow_type_mean_200m
fl_streamorde_sum_200m
fl_streamorde_count_200m
fl_streamorde_mean_200m
fl_intephem_sum_200m
fl_intephem_count_200m
fl_intephem_mean_200m
fl_startflag_sum_200m
fl_startflag_count_200m
fl_startflag_mean_200m
fl_divergence_sum_200m
fl_divergence_count_200m
fl_divergence_mean_2

In [12]:
import pickle

# use protocol 3 for backwards compatibility with Python 3.6 on AWS
pickle.dump(df_merged, open("2021.03.09_ssurgo_nhd_merged","wb"), protocol=3)
df_merged.to_csv("2021.03.09_ssurgo_nhd_merged.csv")

In [13]:

pd.read_pickle("2021.03.09_ssurgo_nhd_merged").columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'jurisdiction_type', 'da_number',
       'district', 'project_name', 'longitude', 'latitude',
       'date_issued_or_denied', 'rha_determination',
       ...
       'fl_length_count_2500m', 'fl_length_mean_2500m', 'hydclprs',
       'aws025wta', 'drclassdcd', 'slopegradd', 'wtdepannmi', 'flodfreqdc',
       'pondfreqpr', 'wtdepaprju'],
      dtype='object', length=132)

In [14]:
pd.read_csv("2021.03.09_ssurgo_nhd_merged.csv")

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,jurisdiction_type,da_number,district,project_name,longitude,latitude,date_issued_or_denied,...,fl_length_count_2500m,fl_length_mean_2500m,hydclprs,aws025wta,drclassdcd,slopegradd,wtdepannmi,flodfreqdc,pondfreqpr,wtdepaprju
0,0,0,0,RAPANOS,LRB-1983-10120,Buffalo,Trade-A-Yacht (Hibiscus Harbor - Union Springs...,-76.70773,42.85821,06/19/2020,...,1,NaN,100.0,7.40,Very poorly drained,1.0,0.0,None,80.0,0.0
1,1,1,1,RAPANOS,LRB-1985-69031,Buffalo,"POOLEY, MARK A.",-75.85524,43.15230,07/07/2016,...,1,NaN,0.0,0.00,NaN,0.0,0.0,NaN,0.0,0.0
2,2,2,2,RAPANOS,LRB-1986-99614,Buffalo,"Bellamy, Michael (previous: MACKO, JOHN)",-78.04046,42.68911,10/12/2017,...,1,NaN,0.0,0.00,NaN,0.0,0.0,NaN,0.0,0.0
3,3,3,3,RAPANOS,LRB-1990-97632,Buffalo,WESTWOOD COUNTRY CLUB,-78.77134,42.97994,06/28/2016,...,2,NaN,5.0,2.25,Moderately well drained,2.0,54.0,None,0.0,54.0
4,4,4,4,RAPANOS,LRB-1991-98611,Buffalo,MODERN LANDFILL INCORPORATED,-78.97142,43.21616,03/22/2016,...,0,0.0,93.0,5.20,Poorly drained,1.0,0.0,None,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14614,14614,14614,14614,RAPANOS,SWL-2015-00176,Little Rock,Pine Bluff Arsenal: Building Demolition,-92.09857,34.32060,05/20/2016,...,1,NaN,10.0,5.50,Somewhat poorly drained,2.0,31.0,None,0.0,31.0
14615,14615,14615,14615,RAPANOS,SWL-2016-00403,Little Rock,City of Marshall - Waterline Replacement,-92.63156,35.90693,01/19/2018,...,0,0.0,0.0,3.85,Well drained,5.0,0.0,None,0.0,0.0
14616,14616,14616,14616,RAPANOS,SWT-2014-00848-rbh,Tulsa,Sofidel/AEP/Terracon Project 04147070-1 JD alo...,-95.55320,36.12439,01/30/2018,...,2,NaN,0.0,3.61,Somewhat excessively drained,15.0,0.0,None,0.0,0.0
14617,14617,14617,14617,RAPANOS,SWT-2016-00539,Tulsa,"Proposed Walmart Development 45 acre site, NW ...",-97.58169,35.65544,11/17/2016,...,0,0.0,0.0,4.45,Well drained,4.0,0.0,None,0.0,0.0


In [16]:
df_merged.wb_ftype_str_list_200m

0                []
1        [LakePond]
2        [LakePond]
3                []
4                []
            ...    
14614            []
14615            []
14616            []
14617            []
14618            []
Name: wb_ftype_str_list_200m, Length: 14619, dtype: object